In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 6.4e-07}
band = [9]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.041845                    1 -0.003631
0.000750        2 -0.038902                    2 -0.002077
0.001052        3 -0.032411                    3  0.001772
0.001476        4 -0.025230                    4  0.005858
0.002070        5 -0.017451                    5  0.010369
0.002904        6 -0.008976                    6  0.015313
0.004074        7  0.000196                    7  0.020700
0.005714        8  0.010092                    8  0.026625
0.008015        9  0.020738                    9  0.033095
0.011243       10  0.032146                   10  0.040154
0.015771       11  0.044323                   11  0.047844
0.022122       12  0.057266                   12  0.056208
0.031031       13  0.070887                   13  0.065259
0.043528       14  0.083204                   14  0.073871
0.061057       15  0.092166                   15  0.080930
0.085645       16  0.099181                   16  0.087327
0.120136       17  0.105714                   17  0.093976
0.168516       18  0.111773                   18  0.100911
0.236378       19  0.117452                   19  0.108091
0.331549       20  0.123134                   20  0.115554
0.465100       21  0.128584                   21  0.122468
0.652400       22  0.115343                   22  0.116215
0.915100       23  0.081178                   23  0.093826
1.283650       24  0.050313                   24  0.069892
1.800600       25  0.029761                   25  0.049512
2.525700       26  0.017015                   26  0.032448
3.542800       27  0.009335                   27  0.018647
4.969550       28  0.005831                   28  0.009287
6.970850       29  0.005202                   29  0.004481
9.778100       30  0.004891                   30  0.002109
13.715850      31  0.003986                   31  0.000768
19.239350      32  0.002620                   32 -0.000026
26.987250      33  0.002046                   33  0.000210
37.855300      34  0.002471                   34  0.001116
53.100050      35  0.002708                   35  0.001445
73.887500      36  0.002688                   36  0.001037
97.662500      37  0.002392                   37  0.000059
121.437500     38  0.001402                   38 -0.001555
145.212500     39 -0.000024                   39 -0.003519
168.987500     40 -0.001451                   40 -0.005451
192.762500     41 -0.002975                   41 -0.007406
216.537500     42 -0.004671                   42 -0.009440
240.312500     43 -0.006633                   43 -0.011584
264.087500     44 -0.009014                   44 -0.013867
287.862500     45 -0.011796                   45 -0.016108
311.637500     46 -0.013170                   46 -0.016971
335.412500     47 -0.011985                   47 -0.015704
359.187500     48 -0.009822                   48 -0.013389
382.962500     49 -0.007722                   49 -0.010866
406.737500     50 -0.005620                   50 -0.008197
430.512500     51 -0.003446                   51 -0.005389
454.287500     52 -0.001166                   52 -0.002464
478.062500     53  0.001233                   53  0.000617
501.837500     54  0.003745                   54  0.003788
525.612500     55  0.006353                   55  0.007028
549.387500     56  0.009024                   56  0.010303
573.162500     57  0.011710                   57  0.013564
596.937500     58  0.014356                   58  0.016751
620.712500     59  0.016905                   59  0.019785
644.487500     60  0.019310                   60  0.022674
668.262500     61  0.021545                   61  0.025326
692.037500     62  0.023616                   62  0.027767
715.812500     63  0.025552                   63  0.030018
739.587500     64  0.027403                   64  0.032112
763.362500     65  0.029228                   65  0.034082
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -8.302960  0.000000 -8.302960 -8.133220  9.399408e-08   
0.000624    2     -8.302965  0.000002 -8.302963 -8.133222  1.256226e-06   
0.000876    3     -8.302967  0.000003 -8.302964 -8.133222  1.754934e-06   
0.001229    4     -8.302970  0.000004 -8.302966 -8.133223  2.536203e-06   
0.001723    5     -8.302973  0.000006 -8.302967 -8.133224  3.750191e-06   
0.002417    6     -8.302978  0.000010 -8.302969 -8.133225  5.636820e-06   
0.003391    7     -8.302984  0.000015 -8.302970 -8.133226  8.567243e-06   
0.004757    8     -8.302992  0.000022 -8.302970 -8.133227  1.311353e-05   
0.006672    9     -8.303002  0.000034 -8.302967 -8.133228  2.016404e-05   
0.009359    10    -8.303013  0.000053 -8.302961 -8.133229  3.108603e-05   
0.013128    11    -8.303027  0.000081 -8.302946 -8.133228  4.798609e-05   
0.018415    12    -8.303043  0.000125 -8.302919 -8.133224  7.410312e-05   
0.025830    13    -8.303059  0.000191 -8.302868 -8.133215  1.144086e-04   
0.036232    14    -8.303072  0.000291 -8.302781 -8.133196  1.764977e-04   
0.050823    15    -8.303079  0.000442 -8.302637 -8.133163  2.709499e-04   
0.071291    16    -8.303074  0.000660 -8.302414 -8.133108  4.117740e-04   
0.100000    17    -8.303050  0.000974 -8.302076 -8.133019  6.196758e-04   
0.140271    18    -8.302990  0.001418 -8.301572 -8.132877  9.260462e-04   
0.196760    19    -8.302868  0.002044 -8.300824 -8.132652  1.376584e-03   
0.275997    20    -8.302634  0.002912 -8.299722 -8.132298  2.037033e-03   
0.387100    21    -8.302202  0.004101 -8.298101 -8.131741  3.001007e-03   
0.543100    22    -8.301420  0.005695 -8.295725 -8.130871  4.394818e-03   
0.761700    23    -8.300278  0.007540 -8.292738 -8.129687  6.220104e-03   
1.068500    24    -8.299012  0.009225 -8.289788 -8.128350  8.294050e-03   
1.498800    25    -8.297755  0.010532 -8.287223 -8.126965  1.047116e-02   
2.102400    26    -8.296544  0.011448 -8.285096 -8.125581  1.262802e-02   
2.949000    27    -8.295436  0.012047 -8.283389 -8.124282  1.458334e-02   
4.136600    28    -8.294556  0.012480 -8.282076 -8.123198  1.612343e-02   
5.802500    29    -8.293988  0.013063 -8.280925 -8.122413  1.717128e-02   
8.139200    30    -8.293690  0.014205 -8.279485 -8.121916  1.791469e-02   
11.417000   31    -8.293732  0.016146 -8.277586 -8.121834  1.865080e-02   
16.014700   32    -8.294458  0.019043 -8.275415 -8.122603  1.983832e-02   
22.464000   33    -8.296625  0.023212 -8.273413 -8.125033  2.224831e-02   
31.510500   34    -8.300943  0.029722 -8.271221 -8.129952  2.739222e-02   
44.200100   35    -8.307844  0.040338 -8.267506 -8.138193  3.731142e-02   
62.000000   36    -8.318540  0.056744 -8.261795 -8.151746  5.391063e-02   
85.775000   37    -8.334332  0.080107 -8.254225 -8.173137  7.822236e-02   
109.550000  38    -8.351789  0.104302 -8.247488 -8.198241  1.034919e-01   
133.325000  39    -8.371661  0.128122 -8.243539 -8.227795  1.286647e-01   
157.100000  40    -8.394773  0.151167 -8.243606 -8.262443  1.534005e-01   
180.875000  41    -8.421405  0.173714 -8.247691 -8.302321  1.779257e-01   
204.650000  42    -8.451915  0.195844 -8.256070 -8.347643  2.023888e-01   
228.425000  43    -8.486814  0.217589 -8.269226 -8.398720  2.268760e-01   
252.200000  44    -8.526863  0.238957 -8.287906 -8.455860  2.513894e-01   
275.975000  45    -8.573263  0.259970 -8.313293 -8.519437  2.759098e-01   
299.750000  46    -8.627560  0.281045 -8.346515 -8.589583  3.006858e-01   
323.525000  47    -8.688685  0.305079 -8.383606 -8.664432  3.277330e-01   
347.300000  48    -8.753017  0.335657 -8.417360 -8.740840  3.599084e-01   
371.075000  49    -8.818410  0.373389 -8.445021 -8.816984  3.983409e-01   
394.850000  50    -8.884212  0.417442 -8.466769 -8.892177  4.429298e-01   
418.625000  51    -8.949903  0.467306 -8.482598 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')